---
title: Create Tables
authors:
  - name: Dimitri Yatsenko
date: 2025-10-31
---

# Tables: The Foundation of Data Integrity

In the [previous chapter](012-integrity.md), we learned that relational databases excel at enforcing **data integrity constraints**. Tables are where these constraints come to life through:

1. **Domain Integrity** — Data types restrict values to valid ranges
2. **Completeness** — Required vs. optional attributes ensure necessary data is present
3. **Entity Integrity** — Primary keys uniquely identify each record (covered in [Primary Keys](020-primary-key.md))
4. **Referential Integrity** — Foreign keys enforce relationships (covered in [Foreign Keys](030-foreign-keys.ipynb))

This chapter shows how to declare tables in DataJoint with proper data types and attribute specifications that enforce these constraints automatically.

```{admonition} Learning Objectives
:class: note

By the end of this chapter, you will:
- Declare tables with the `@schema` decorator
- Specify attributes with appropriate data types
- Distinguish between primary key and dependent attributes
- Insert, view, and delete data
- Understand how table structure enforces integrity
```

# Prerequisites: Schema Declaration

Before creating tables, we must declare a schema object that provides a namespace for our tables (see [Create Schemas](010-schema.ipynb)):

In [ ]:
import datajoint as dj

# Define the schema namespace
schema = dj.Schema('tutorial')

# Table Declaration Syntax

To declare a table in DataJoint, define a Python class with:
1. **Schema decorator** — `@schema` associates the table with the schema
2. **Base class** — Inherit from `dj.Manual`, `dj.Lookup`, `dj.Imported`, or `dj.Computed`
3. **Definition string** — Multi-line string specifying attributes and types

Here's a basic example:

In [ ]:
@schema
class Experiment(dj.Manual):
    definition = """
    experiment_id: int      # Unique identifier for each experiment
    ---
    experiment_name: varchar(64)   # Name of the experiment
    experiment_date: date          # Date when the experiment was conducted
    """

## Anatomy of a Table Definition

**Table Name:** The Python class name (`Experiment`) becomes the table name in the database.
- Convention: Singular, CamelCase (e.g., `RecordingSession`, not `recording_sessions`)
- No underscores in multi-word names

**Attribute Specification:** Each line defines one attribute (column):
```
attribute_name : datatype  # optional comment
```

**Primary Key Separator:** The line `---` divides:
- **Above** `---`: Primary key attributes (jointly uniquely identify each entity)
- **Below** `---`: Dependent attributes (describe the entity)

**Comments:** Text after `#` documents the attribute's meaning

```{admonition} Why This Structure?
:class: tip

This declarative syntax makes integrity constraints explicit:
- **Data types** enforce domain integrity
- **Primary key position** enforces entity integrity
- **Required attributes** enforce completeness (see [Default Values](017-default-values.ipynb) for optional attributes)
```

# Data Types: Enforcing Domain Integrity

DataJoint uses MySQL data types to enforce **domain integrity** — ensuring values are within valid ranges. Here are the most commonly used types:

## Numeric Types

**Integer Types** (for whole numbers):
- `tinyint` — Range: -128 to 127 (or 0 to 255 if `unsigned`)
- `smallint` — Range: -32,768 to 32,767 (or 0 to 65,535 if `unsigned`)
- `int` — Range: -2.1B to 2.1B (or 0 to 4.3B if `unsigned`)
- `bigint` — Range: -9.2E18 to 9.2E18 (or 0 to 1.8E19 if `unsigned`)

**Floating-Point Types** (for approximate decimal numbers):
- `float` — 32-bit floating point (~7 decimal digits precision)
- `double` — 64-bit floating point (~15 decimal digits precision)

**Fixed-Point Type** (for exact decimal numbers):
- `decimal(M, D)` — M total digits, D after decimal point
  - Example: `decimal(10, 2)` for monetary values like 12345678.90
  - Can be `unsigned` for non-negative values only

```{admonition} Choosing Numeric Types
:class: tip

- **Storage efficiency**: Use `tinyint` for small counts (0-255), not `int`
- **Precision**: Use `decimal` for financial data, `float/double` for measurements
- **Unsigned**: Add `unsigned` when values are never negative (e.g., counts, IDs)
```

## Temporal Types

- `date` — Date only: `YYYY-MM-DD` (e.g., `2024-10-31`)
- `time` — Time only: `HH:MM:SS` (e.g., `14:30:00`)
- `datetime` — Date and time: `YYYY-MM-DD HH:MM:SS`
- `timestamp` — Unix timestamp (seconds since 1970-01-01, auto-updates)

## String Types

- `char(M)` — Fixed-length string, padded to M characters (max 255)
  - Use for: Fixed-length codes (e.g., `char(2)` for US state codes)
- `varchar(M)` — Variable-length string, up to M characters (max 65,535)
  - Use for: Names, descriptions, free text
- `enum('val1', 'val2', ...)` — Restricts values to specified options
  - Use for: Categorical data with known set (e.g., experimental conditions)

```{admonition} String Type Selection
:class: tip

- `enum` provides strongest domain integrity for categorical data
- `varchar(50)` is more flexible than `char(50)` and saves space
- Choose `varchar` length based on expected maximum, not arbitrary large values
```

## Example with Multiple Data Types

In [ ]:
@schema
class Mouse(dj.Manual):
    definition = """
    mouse_id: int unsigned          # Unique mouse identifier (1-4B range)
    ---
    mouse_name: varchar(50)         # Mouse name or nickname
    date_of_birth: date             # Birth date
    sex: enum('M', 'F', 'U')        # Sex: Male, Female, or Unknown
    weight: decimal(5,2) unsigned   # Weight in grams (0.00-999.99)
    """

This definition enforces:
- `mouse_id` must be a non-negative integer
- `mouse_name` cannot exceed 50 characters
- `date_of_birth` must be a valid date
- `sex` can only be 'M', 'F', or 'U'
- `weight` must be a non-negative decimal with 2 decimal places

The database automatically rejects any insert that violates these constraints.

```{seealso}
For complete MySQL data type documentation, see [MySQL Data Types](https://dev.mysql.com/doc/refman/8.4/en/data-types.html).
```

# Inserting Data

Once a table is declared, insert data using:

**`insert1(record)`** — Insert a single record (dictionary)

In [ ]:
Experiment.insert1({
    'experiment_id': 1,
    'experiment_name': 'Spatial Navigation Task',
    'experiment_date': '2024-10-31'
})

**`insert(records)`** — Insert multiple records (list of dictionaries)

In [ ]:
Experiment.insert([
    {'experiment_id': 2, 'experiment_name': 'Fear Conditioning', 'experiment_date': '2024-11-01'},
    {'experiment_id': 3, 'experiment_name': 'Object Recognition', 'experiment_date': '2024-11-02'}
])

```{admonition} Integrity Enforcement at Insert
:class: important

The database validates each insert:
- **Domain integrity**: Rejects invalid data types (e.g., string in `int` field)
- **Completeness**: Rejects missing required attributes
- **Entity integrity**: Rejects duplicate primary keys (see [Primary Keys](020-primary-key.md))

Violations raise immediate errors — no invalid data ever enters the database.
```

# Viewing Table Contents

## View Table Structure

Reference the table class to see its definition:

In [ ]:
Experiment

## View Table Data

Call the table class (with parentheses) to see its contents:

In [ ]:
Experiment()  # Shows all records in table

Output displays:
- **Asterisk (*)** marks primary key attributes
- All attribute values for each record
- Total record count

```{note}
The `()` syntax creates a **query object** representing all table records. You'll learn to create more complex queries in the [Queries](../queries/) section.
```

# Deleting Records

## Delete Specific Records

Use the **restriction operator** `&` with primary key values:

In [ ]:
# Delete experiment with ID 1
(Experiment & {'experiment_id': 1}).delete()

You'll be prompted to confirm before deletion is committed.

## Delete All Records

Call `.delete()` on the table class:

In [ ]:
Experiment.delete()  # Prompts for confirmation

```{warning}
Deletion is **permanent** and will cascade to dependent tables (see [Foreign Keys](030-foreign-keys.ipynb)). Always verify what will be deleted before confirming.
```

# Dropping Tables

**Dropping a table** permanently removes it and all its data from the database:

In [ ]:
Experiment.drop()  # Prompts for confirmation

```{warning}
**This action is irreversible!**

- Removes table structure and all data
- Cascades to dependent tables if they exist
- Requires kernel restart after dropping

Always back up critical data before dropping tables.
```

For information on dropping entire schemas, see [Create Schemas](010-schema.ipynb).

# Table Base Classes

DataJoint provides four base classes for different data management patterns:

| Base Class | Purpose | When to Use |
|------------|---------|-------------|
| `dj.Manual` | Manually entered data | Subject info, experimental protocols |
| `dj.Lookup` | Reference data, rarely changes | Equipment lists, parameter sets |
| `dj.Imported` | Data imported from external files | Raw recordings, behavioral videos |
| `dj.Computed` | Derived from other tables | Spike sorting results, analyses |

We'll explore `Imported` and `Computed` tables in the [Computation](../60-computation/) section.

```{seealso}
- [Lookup Tables](018-lookup-tables.ipynb) — Managing reference data
- [Computation](../60-computation/010-computation.ipynb) — Automated data processing
```

# Summary

Tables are the fundamental building blocks where data integrity is enforced:

1. **Table declarations** specify structure using the `@schema` decorator and `definition` string
2. **Data types** enforce domain integrity by restricting values to valid ranges
3. **Primary keys** (above `---`) enforce entity integrity through unique identification
4. **Required attributes** enforce completeness by ensuring necessary data is present
5. **DataJoint operations** (`insert`, `delete`, `drop`) respect these integrity constraints

```{admonition} Next Steps
:class: tip

Now that you understand table structure and data types, the next chapters explore:
- **[Default Values](017-default-values.ipynb)** — Making attributes optional
- **[Primary Keys](020-primary-key.md)** — Strategies for unique entity identification
- **[Foreign Keys](030-foreign-keys.ipynb)** — Linking tables through relationships
```